# Network analysis - World of Warcraft

## Get data from Battle.net

#### Setup connection

In [1]:
with open('battle_net_api.key', 'r') as f:
    key = f.read()

#### Import API Key

In [109]:
import battlenet_api
reload(battlenet_api)

<module 'battlenet_api' from 'battlenet_api.py'>

In [110]:
bn = battlenet_api.BattleNet(public_key=key)

#### Get data for a PVE Realm

In [111]:
realms = bn.realms()

In [105]:
print "Number of Realms: %d" % len(realms)

Number of Realms: 246


In [6]:
realms_pve = [r for r in realms if r['type']=='pve']

In [7]:
print "Number of PVE Realms: %d" % len(realms_pve)

Number of PVE Realms: 117


In [52]:
realm = realms_pve[0]

In [53]:
print "Selected Realm: %s" % realm['name']

Selected Realm: Aerie Peak


#### Get data for Challenges in the selected Realm

In [54]:
challenges = bn.challenges(realm['name'])

In [55]:
print "Number of Challenges: %d" % len(challenges)

Number of Challenges: 17


In [56]:
groups = challenges[0]['groups']

In [57]:
print "Number of groups per Challenge: %d" % len(groups)

Number of groups per Challenge: 100


#### Figure out distribution of characters in groups

In [58]:
import numpy as np

In [93]:
names = []
for challenge in challenges:
    num_members = []  # list of num. of members
    for group in challenge['groups']:
        members = group['members']
        num_members += [len(members)]
        for m in members:
            if m.has_key('character'):
                 names += [m['character']['name']]
#         print ', '.join(names)

    print "Challenge %s group members: %0.2f average and %0.2f std" % \
    (challenge['map']['name'], np.mean(num_members), np.std(num_members))

Challenge Auchindoun group members: 5.00 average and 0.00 std
Challenge Bloodmaul Slag Mines group members: 5.00 average and 0.00 std
Challenge Gate of the Setting Sun group members: 5.00 average and 0.00 std
Challenge Grimrail Depot group members: 5.00 average and 0.00 std
Challenge Iron Docks group members: 5.00 average and 0.00 std
Challenge Mogu'shan Palace group members: 5.00 average and 0.00 std
Challenge Scarlet Halls group members: 5.00 average and 0.00 std
Challenge Scarlet Monastery group members: 5.00 average and 0.00 std
Challenge Scholomance group members: 5.00 average and 0.00 std
Challenge Shado-Pan Monastery group members: 5.00 average and 0.00 std
Challenge Shadowmoon Burial Grounds group members: 5.00 average and 0.00 std
Challenge Siege of Niuzao Temple group members: 5.00 average and 0.00 std
Challenge Skyreach group members: 5.00 average and 0.00 std
Challenge Stormstout Brewery group members: 5.00 average and 0.00 std
Challenge Temple of the Jade Serpent group mem

In [60]:
print "Number of unique characters: %d" % len(set(names))

Number of unique characters: 1057


#### Create a network of characters

In [61]:
import networkx as nx

In [62]:
import algos
reload(algos)

<module 'algos' from 'algos.pyc'>

In [63]:
g = algos.generate_team_graph(challenges)

## Algebraic Connectivity of network and performance

#### Calculate correlation between algebraic connectivity and performance

In [64]:
from scipy.stats import pearsonr

In [65]:
ac, time = algos.get_metrics(challenges, g)

In [66]:
ac = np.array(ac)
time = np.array(time)
r, p = pearsonr(ac, time)

In [67]:
print "Algebraic Connectivity vs. challenge completion time: Pearson coeff = %0.2f, p-value = %0.6f" % (r, p)

Algebraic Connectivity vs. challenge completion time: Pearson coeff = -0.12, p-value = 0.000002


#### Calculate correlation between algebraic connectivity and performance for all PVE Realms

In [82]:
import time

In [118]:
corr = []  # list of tuples [(realm name, corr coeff, p value), ...]
for realm in realms_pve:
    realm_name = realm['name']
#     print realm_name
    try:
        challenges = bn.challenges(realm_name)
        g = algos.generate_team_graph(challenges)
        ac, c_time = algos.get_metrics(challenges, g)
        ac = np.array(ac)
        c_time = np.array(c_time)
        r, p = pearsonr(ac, c_time)
        corr += [(realm_name, r, p)]
        print "Realm: %s, \t Pearson coeff = %0.2f, \t p-value = %0.6f" % (realm_name, r, p)    
    except:
        print "Error with '%s'" % realm_name
    time.sleep(1)
#     break

Realm: Aerie Peak, 	 Pearson coeff = -0.12, 	 p-value = 0.000002
Realm: Aggramar, 	 Pearson coeff = -0.09, 	 p-value = 0.000965
Realm: Alexstrasza, 	 Pearson coeff = -0.05, 	 p-value = 0.046314
Realm: Alleria, 	 Pearson coeff = -0.08, 	 p-value = 0.001852
Realm: Aman'Thul, 	 Pearson coeff = -0.17, 	 p-value = 0.000000
Realm: Antonidas, 	 Pearson coeff = 0.15, 	 p-value = 0.000000
Error with 'Anvilmar'
Error with 'Arathor'
Error with 'Area 52'
Error with 'Arygos'
Error with 'Azjol-Nerub'
Realm: Azuremyst, 	 Pearson coeff = -0.06, 	 p-value = 0.013243
Error with 'Baelgun'
Realm: Blackhand, 	 Pearson coeff = -0.06, 	 p-value = 0.026009
Error with 'Blade's Edge'
Realm: Bladefist, 	 Pearson coeff = -0.06, 	 p-value = 0.028214
Error with 'Bloodhoof'
Realm: Borean Tundra, 	 Pearson coeff = 0.02, 	 p-value = 0.537992
Error with 'Bronzebeard'
Realm: Caelestrasz, 	 Pearson coeff = -0.25, 	 p-value = 0.000000
Realm: Cairne, 	 Pearson coeff = -0.05, 	 p-value = 0.041315
Error with 'Cenarius'
Error

In [126]:
for realm_name, r, p in corr:
    print "Realm: %s,  \t Pearson coeff = %0.2f,  \t p-value = %0.6f" % (realm_name, r, p)    

Realm: Aerie Peak,  	 Pearson coeff = -0.12,  	 p-value = 0.000002
Realm: Aggramar,  	 Pearson coeff = -0.09,  	 p-value = 0.000965
Realm: Alexstrasza,  	 Pearson coeff = -0.05,  	 p-value = 0.046314
Realm: Alleria,  	 Pearson coeff = -0.08,  	 p-value = 0.001852
Realm: Aman'Thul,  	 Pearson coeff = -0.17,  	 p-value = 0.000000
Realm: Antonidas,  	 Pearson coeff = 0.15,  	 p-value = 0.000000
Realm: Azuremyst,  	 Pearson coeff = -0.06,  	 p-value = 0.013243
Realm: Blackhand,  	 Pearson coeff = -0.06,  	 p-value = 0.026009
Realm: Bladefist,  	 Pearson coeff = -0.06,  	 p-value = 0.028214
Realm: Borean Tundra,  	 Pearson coeff = 0.02,  	 p-value = 0.537992
Realm: Caelestrasz,  	 Pearson coeff = -0.25,  	 p-value = 0.000000
Realm: Cairne,  	 Pearson coeff = -0.05,  	 p-value = 0.041315
Realm: Darrowmere,  	 Pearson coeff = -0.08,  	 p-value = 0.002386
Realm: Dawnbringer,  	 Pearson coeff = -0.19,  	 p-value = 0.000000
Realm: Doomhammer,  	 Pearson coeff = -0.03,  	 p-value = 0.170234
Realm